In [2]:
import torch
import load_dataset
import torchaudio
import torchaudio.functional as F
# load the cnn model with pytorch
path = 'models/model_base.ckpt'
model = load_dataset.CNN()
model.load_state_dict(torch.load(path))
# load the audio file
image = load_dataset.load_dataset('../../sound-propagation-simulation/dataset_real_data/v3/test/')
#image = load_dataset.load_dataset('patch-attack-data/')
for i in range(len(image['waveforms'])):
    waveform = image['waveforms'][i]
    label = image['labels'][i]
    print('true',label,end=' prediction ')
    labels_to_int = {0: 'right_to_left', 1: 'left_to_right', 2: 'multiple_cars', 3: 'no_car'}
    # convert the audio file to spectrogram
    spectrogram = F.spectrogram(waveform, pad=0, window=torch.hann_window(1000), n_fft=1000, hop_length=1280, win_length=1000, power=2.0, normalized=True)
    # add a dimension to the spectrogram
    spectrogram = spectrogram.unsqueeze(0)
    model.eval()
    # predict the label
    output = model(spectrogram)
    # get the index of the highest value
    a, predicted = torch.max(output[0], 1)
    print(labels_to_int[predicted.item()], a)

number of label: 4
true left_to_right prediction no_car tensor([3.7904], grad_fn=<MaxBackward0>)
true left_to_right prediction left_to_right tensor([2.7366], grad_fn=<MaxBackward0>)
true left_to_right prediction left_to_right tensor([-0.1715], grad_fn=<MaxBackward0>)
true left_to_right prediction no_car tensor([0.7648], grad_fn=<MaxBackward0>)
true left_to_right prediction left_to_right tensor([0.9868], grad_fn=<MaxBackward0>)
true left_to_right prediction left_to_right tensor([11.9013], grad_fn=<MaxBackward0>)
true left_to_right prediction no_car tensor([2.7091], grad_fn=<MaxBackward0>)
true left_to_right prediction no_car tensor([0.7435], grad_fn=<MaxBackward0>)
true left_to_right prediction right_to_left tensor([2.5327], grad_fn=<MaxBackward0>)
true left_to_right prediction left_to_right tensor([2.3219], grad_fn=<MaxBackward0>)
true left_to_right prediction right_to_left tensor([-1.5979], grad_fn=<MaxBackward0>)
true left_to_right prediction left_to_right tensor([-3.1029], grad_fn=<

In [3]:
# adversarial attack
# to do adversarial attack, we need to find the gradient of the input with respect to the output
# we need to find the gradient of the input with respect to the output

gradient = torch.autograd.grad(output[0][0][predicted], spectrogram, retain_graph=True)

# then we need to add the gradient to the input

spectrogram = spectrogram + gradient[0]

# then we need to predict the label again

output = model(spectrogram)

# then we need to get the index of the highest value

a, predicted = torch.max(output[0], 1)

# then we need to print the label

print(labels_to_int[predicted.item()], a)

RuntimeError: One of the differentiated Tensors does not require grad